<a href="https://colab.research.google.com/github/Tharmika2002/Ride-Cancellation-Model-DataX/blob/main/Ride_Cancellation_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DataX — Ride Cancellation Prediction
**Notebook:** Ride-Cancellation-Model.ipynb   
**Author:** DataX Team

In [1]:
# === Setup ===
# !pip install pandas matplotlib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 160)


In [2]:
# === Paths / Config ===

PATH = '/content/drive/MyDrive/Bengaluru Ola.csv'
TARGET_COL = "Booking Status"
RANDOM_STATE = 42


In [3]:
# === Load CSV  ===

# Start with df = None, so we know if the file was successfully loaded later
df = None

# Try reading the file using different common encodings
# (sometimes CSVs are saved in utf-8, utf-8-sig, or latin-1 formats)
for enc in ["utf-8", "utf-8-sig", "latin-1"]:
    try:
        # Try to read the CSV with the current encoding
        df = pd.read_csv(PATH, encoding=enc)
        print(f"Successfully loaded with encoding: {enc}")
        break  # If it works, stop trying others
    except Exception as e:
        # If it fails, show the error and try the next encoding
        print(f"Failed with {enc}: {e}")

# Safety check: if df is still None, it means no encoding worked
assert df is not None, "Could not read CSV — check PATH or encoding."

# If we reach here, the dataset is successfully loaded
print("Loaded dataset.")
print("Shape (rows, columns):", df.shape)

# Show the first 5 rows as a quick preview
df.head(5)


Successfully loaded with encoding: utf-8
Loaded dataset.
Shape (rows, columns): (49999, 21)


,Date,Time,Booking ID,Booking Status,Customer ID,Vehicle Type,Pickup Location,Drop Location,Avg VTAT,Avg CTAT,Cancelled by Customer,Reason for Cancelling by Customer,Cancelled Rides by Driver,Reason for Cancelling by Driver,Incomplete Rides,Incomplete Rides Reason,Booking Value,Payment Method,Ride Distance,Driver Ratings,Customer Rating
0,28/01/2024,06:00:00,CNR1721175,Success,329258,Auto,Area-3,Area-2,5.42,18.46,0,NaN,0,NaN,0,NaN,868.06,Wallet,28.5,4.4,4.4
1,26/01/2024,03:00:00,CNR2871422,Cancelled by Driver,201414,Mini,Area-7,Area-6,NaN,NaN,0,NaN,1,The customer was coughing/sick,0,NaN,NaN,NaN,NaN,NaN,NaN
2,15/01/2024,16:00:00,CNR6875935,Cancelled by Driver,301629,Bike,Area-40,Area-24,NaN,NaN,0,NaN,1,Personal & Car related issues,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2/1/2024,22:00:00,CNR6798834,Cancelled by Driver,319684,Prime Sedan,Area-11,Area-24,NaN,NaN,0,NaN,1,The customer was coughing/sick,0,NaN,NaN,NaN,NaN,NaN,NaN
4,30/01/2024,22:00:00,CNR9661713,Incomplete,330283,Bike,Area-41,Area-45,NaN,NaN,0,NaN,0,NaN,1,Other Issue,NaN,NaN,NaN,NaN,NaN
